In [1]:
import pandas as pd
import numpy as np
import pickle
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import math
from scipy.fftpack import fft, fftfreq
import string
import tensorflow as tf
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, TimeDistributed, Dropout, Conv2D, Conv1D, MaxPooling1D
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorboard.plugins.hparams import api as hp
import pydot
tf.keras.backend.clear_session()
#%load_ext tensorboard

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from ipynb.fs.full.formata_dados import *
from ipynb.fs.full.processa_dados import *

In [4]:
tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
) 

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [5]:
#Importa os dados formatados
with open("../data/Pickles/dados_ok.pkl", "rb") as dados_ok:
        dados_ok = pickle.load(dados_ok)[0]
with open("../data/Pickles/dados_agua_ok.pkl", "rb") as dados_agua_ok:
        dados_agua_ok = pickle.load(dados_agua_ok)[0]
with open("../data/Pickles/dados_aspartame_ok.pkl", "rb") as dados_aspartame_ok:
        dados_aspartame_ok = pickle.load(dados_aspartame_ok)[0]
with open("../data/Pickles/dados_sucralose_ok.pkl", "rb") as dados_sucralose_ok:
        dados_sucralose_ok = pickle.load(dados_sucralose_ok)[0]
with open("../data/Pickles/dados_acucar_ok.pkl", "rb") as dados_acucar_ok:
        dados_acucar_ok = pickle.load(dados_acucar_ok)[0]
with open("../data/Pickles/todos_os_dados_originais.pkl", "rb") as todos_os_dados:
        dados_originais = pickle.load(todos_os_dados)[0]

In [6]:
#remove amplitudes consideradas como ruido na serie
dados_sem_ruidos = remove_ruidos(dados_ok,80) #80 - substituir valores com amplitude acima de 80
with open("../data/Pickles/dados_ok.pkl", "rb") as dados_ok:
    dados_ok = pickle.load(dados_ok)[0]

In [7]:
#Carrega/salva dados
gerar_dados = True
if gerar_dados == False:
    #referencias das bebidas
    agua = 0
    acucar = 1
    aspartame = 2
    sucralose = 3
    dict_output = enumera_dicionario(['agua', 'acucar', 'aspartame', 'sucralose'])
    dados_sem_ruidos_separados_por_output = separa_dados_por_output(dados_sem_ruidos,dict_output)
    with open("../data/Pickles/dados_semruido_ok.pkl", "wb") as dados_ok_semruidos:
            pickle.dump([dados_sem_ruidos], dados_ok_semruidos)
    with open("../data/Pickles/dados_semruido_agua_ok.pkl", "wb") as dados_agua_ok:
            pickle.dump([dados_sem_ruidos_separados_por_output['agua']], dados_agua_ok)
    with open("../data/Pickles/dados_semruido_aspartame_ok.pkl", "wb") as dados_aspartame_ok:
            pickle.dump([dados_sem_ruidos_separados_por_output['aspartame']], dados_aspartame_ok)
    with open("../data/Pickles/dados_semruido_sucralose_ok.pkl", "wb") as dados_sucralose_ok:
            pickle.dump([dados_sem_ruidos_separados_por_output['sucralose']], dados_sucralose_ok)
    with open("../data/Pickles/dados_semruido_acucar_ok.pkl", "wb") as dados_acucar_ok:
            pickle.dump([dados_sem_ruidos_separados_por_output['acucar']], dados_acucar_ok)
else:
    with open("../data/Pickles/dados_semruido_ok.pkl", "rb") as dados_ok_semruidos:
            dados_semruido_ok = pickle.load(dados_ok_semruidos)[0]
    with open("../data/Pickles/dados_semruido_agua_ok.pkl", "rb") as dados_agua_ok:
            dados_semruido_agua_ok= pickle.load(dados_agua_ok)[0]
    with open("../data/Pickles/dados_semruido_aspartame_ok.pkl", "rb") as dados_aspartame_ok:
            dados_semruido_aspartame_ok = pickle.load(dados_aspartame_ok)[0]
    with open("../data/Pickles/dados_semruido_sucralose_ok.pkl", "rb") as dados_sucralose_ok:
            dados_semruido_sucralose_ok = pickle.load(dados_sucralose_ok)[0]
    with open("../data/Pickles/dados_semruido_acucar_ok.pkl", "rb") as dados_acucar_ok:
            dados_semruido_acucar_ok = pickle.load(dados_acucar_ok)[0]

In [8]:
# gerar os vetores de input e output dos dados sem ruidos

X_all = dados_sem_ruidos[:,:-4]
y_all = dados_sem_ruidos[:,-4:]
X_all_agua = dados_semruido_agua_ok[:,:-4]
y_all_agua = dados_semruido_agua_ok[:,-4:]
X_all_acucar = dados_semruido_acucar_ok[:,:-4]
y_all_acucar = dados_semruido_acucar_ok[:,-4:]
X_all_aspartame = dados_semruido_aspartame_ok[:,:-4]
y_all_aspartame = dados_semruido_aspartame_ok[:,-4:]
X_all_sucralose = dados_semruido_sucralose_ok[:,:-4]
y_all_sucralose = dados_semruido_sucralose_ok[:,-4:]

In [9]:
def get_best(X_test, y_test, best_model):
    doce_certo, agua_certo = np.linspace(0, 1, num=101), np.linspace(0, 1, num=101)
    for i, j in enumerate(doce_certo):
        agua_certo[i], doce_certo[i] = calc_preds(j, X_test, y_test, best_model)
    b = agua_certo + doce_certo
    return b.argmax(), doce_certo[b.argmax()], agua_certo[b.argmax()], [agua_certo, doce_certo]

# Aqui começa o algoritmo

In [10]:
quant_agua, quant_acucar, quant_aspartame, quant_sucralose = X_all_agua.shape[0], X_all_acucar.shape[0], X_all_aspartame.shape[0], X_all_sucralose.shape[0]

In [11]:
s_agua = int(2*X_all_agua.shape[0]/3)
s_acucar = int(2*X_all_acucar.shape[0]/3)
s_aspartame = int(2*X_all_aspartame.shape[0]/3)
s_sucralose = int(2*X_all_sucralose.shape[0]/3)

X_all_agua_train = X_all_agua[:s_agua, :]
X_all_agua_test = X_all_agua[s_agua:, :]
X_all_acucar_train = X_all_acucar[:s_acucar, :]
X_all_acucar_test = X_all_acucar[s_acucar:, :]
X_all_aspartame_train = X_all_aspartame[:s_aspartame, :]
X_all_aspartame_test = X_all_aspartame[s_aspartame:, :]
X_all_sucralose_train = X_all_sucralose[:s_sucralose, :]
X_all_sucralose_test = X_all_sucralose[s_sucralose:, :]

# X_all_agua_train, X_all_agua_test, _, _ = train_test_split(X_all_agua, y_all_agua, test_size=0.3)
# X_all_acucar_train, X_all_acucar_test, _, _ = train_test_split(X_all_acucar, y_all_acucar, test_size=0.3)
# X_all_aspartame_train, X_all_aspartame_test, _, _ = train_test_split(X_all_aspartame, y_all_aspartame, test_size=0.3)
# X_all_sucralose_train, X_all_sucralose_test, _, _ = train_test_split(X_all_sucralose, y_all_sucralose, test_size=0.3)

window_time = 2
t_inicial = 0
delta_time = 16
n_buckets = 20

X_all_agua_train, y_all_agua_train = multiplica_dados(X_all_agua_train,t_inicial ,t_inicial+delta_time ,512 ,window_time, 0.1, y_all_agua)
X_all_acucar_train, y_all_acucar_train = multiplica_dados(X_all_acucar_train,t_inicial ,t_inicial+delta_time ,512 ,window_time, 0.1, y_all_acucar)
X_all_aspartame_train, y_all_aspartame_train = multiplica_dados(X_all_aspartame_train,t_inicial ,t_inicial+delta_time ,512 ,window_time, 0.1, y_all_aspartame)
X_all_sucralose_train, y_all_sucralose_train = multiplica_dados(X_all_sucralose_train,t_inicial ,t_inicial+delta_time ,512 ,window_time, 0.1, y_all_sucralose)

X_all_agua_test, y_all_agua_test = multiplica_dados(X_all_agua_test,t_inicial ,t_inicial+delta_time ,512 ,window_time, 0.1, y_all_agua)
X_all_acucar_test, y_all_acucar_test = multiplica_dados(X_all_acucar_test,t_inicial ,t_inicial+delta_time ,512 ,window_time, 0.1, y_all_acucar)
X_all_aspartame_test, y_all_aspartame_test = multiplica_dados(X_all_aspartame_test,t_inicial ,t_inicial+delta_time ,512 ,window_time, 0.1, y_all_aspartame)
X_all_sucralose_test, y_all_sucralose_test = multiplica_dados(X_all_sucralose_test,t_inicial ,t_inicial+delta_time ,512 ,window_time, 0.1, y_all_sucralose)

X_all_agua_train = converte_dados_buckets_fft(X_all_agua_train, n_buckets)
X_all_acucar_train = converte_dados_buckets_fft(X_all_acucar_train, n_buckets)
X_all_aspartame_train = converte_dados_buckets_fft(X_all_aspartame_train, n_buckets)
X_all_sucralose_train = converte_dados_buckets_fft(X_all_sucralose_train, n_buckets)

X_all_agua_test = converte_dados_buckets_fft(X_all_agua_test, n_buckets)
X_all_acucar_test = converte_dados_buckets_fft(X_all_acucar_test, n_buckets)
X_all_aspartame_test = converte_dados_buckets_fft(X_all_aspartame_test, n_buckets)
X_all_sucralose_test = converte_dados_buckets_fft(X_all_sucralose_test, n_buckets)

In [12]:
experimento = 'acucar_sucralose'

if experimento == 'agua_acucar':
    X_train = np.concatenate((X_all_agua_train, X_all_acucar_train))
    X_test = np.concatenate((X_all_agua_test, X_all_acucar_test))
    y_train = np.concatenate((y_all_agua_train, y_all_acucar_train))
    y_test = np.concatenate((y_all_agua_test, y_all_acucar_test))
elif experimento == 'agua_aspartame':
    X_train = np.concatenate((X_all_agua_train, X_all_aspartame_train))
    X_test = np.concatenate((X_all_agua_test, X_all_aspartame_test))
    y_train = np.concatenate((y_all_agua_train, y_all_aspartame_train))
    y_test = np.concatenate((y_all_agua_test, y_all_aspartame_test))
elif experimento == 'agua_sucralose':
    X_train = np.concatenate((X_all_agua_train, X_all_sucralose_train))
    X_test = np.concatenate((X_all_agua_test, X_all_sucralose_test))
    y_train = np.concatenate((y_all_agua_train, y_all_sucralose_train))
    y_test = np.concatenate((y_all_agua_test, y_all_sucralose_test))    
elif experimento == 'acucar_sucralose':
    X_train = np.concatenate((X_all_acucar_train, X_all_sucralose_train))
    X_test = np.concatenate((X_all_acucar_test, X_all_sucralose_test))
    y_train = np.concatenate((y_all_acucar_train, y_all_sucralose_train))
    y_test = np.concatenate((y_all_acucar_test, y_all_sucralose_test))
elif experimento == 'acucar_aspartame':
    X_train = np.concatenate((X_all_acucar_train, X_all_aspartame_train))
    X_test = np.concatenate((X_all_acucar_test, X_all_aspartame_test))
    y_train = np.concatenate((y_all_acucar_train, y_all_aspartame_train))
    y_test = np.concatenate((y_all_acucar_test, y_all_aspartame_test, ))
elif experimento == 'aspartame_sucralose':
    X_train = np.concatenate((X_all_aspartame_train, X_all_sucralose_train))
    X_test = np.concatenate((X_all_aspartame_test, X_all_sucralose_test))
    y_train = np.concatenate((y_all_aspartame_train, y_all_sucralose_train))
    y_test = np.concatenate((y_all_aspartame_test, y_all_sucralose_test))

# agua = 0
# acucar = 1
# aspartame = 2
# sucralose = 3

y_train = y_train[:,1]
y_test = y_test[:,1]    
    
if 'agua' in experimento:        # quando tempor Agua, ela é o output 0
    y_train[y_train > 0.1] = 1
    y_test[y_test > 0.1] = 1
elif 'acucar' in experimento:    # quando tempos açúcar (sem agua), ela é o output 1
    y_train[y_train > 1] = 0
    y_test[y_test > 1] = 0
else:
    print("else")
    y_train[y_train == 2] = 0      # aspartame vs sucralose: aspartame=0, sucralose=1
    y_train[y_train == 3] = 1 
    y_test[y_test == 2] = 0
    y_test[y_test == 3] = 1

# X_train = tf.keras.utils.normalize(X_train, axis=-1, order=2)
# X_test = tf.keras.utils.normalize(X_test, axis=-1, order=2)
X_train_len = X_train.shape[0]
X_all = tf.keras.utils.normalize(np.concatenate((X_train, X_test)), axis=-1, order=2)
X_train = X_all[:X_train_len,:]
X_test = X_all[X_train_len:,:]

In [13]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((2940, 20), (2940,), (1680, 20), (1680,))

In [19]:
model = tf.keras.Sequential()
model.add(Dense(16, activation='relu')) # retified linear unit
model.add(Dropout(0.2))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [20]:
model.fit(X_train, y_train, epochs=5, verbose=False, shuffle=True, batch_size=512)

In [21]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              multiple                  336       
_________________________________________________________________
dropout_3 (Dropout)          multiple                  0         
_________________________________________________________________
dense_5 (Dense)              multiple                  272       
_________________________________________________________________
dropout_4 (Dropout)          multiple                  0         
_________________________________________________________________
dense_6 (Dense)              multiple                  1088      
_________________________________________________________________
dropout_5 (Dropout)          multiple                  0         
_________________________________________________________________
dense_7 (Dense)              multiple                 

In [ ]:
tf.keras.utils.plot_model(
    model, to_file='model.png', show_shapes=True, show_layer_names=True
)

[autoreload of tensorflow_estimator.python.estimator.export.export failed: Traceback (most recent call last):
  File "/home/amanda/.local/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/amanda/.local/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 450, in superreload
    update_generic(old_obj, new_obj)
  File "/home/amanda/.local/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 387, in update_generic
    update(a, b)
  File "/home/amanda/.local/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 357, in update_class
    update_instances(old, new)
  File "/home/amanda/.local/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 317, in update_instances
    update_instances(old, new, obj, visited)
  File "/home/amanda/.local/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 317, in update_instances
    update_

[autoreload of tensorflow_estimator.python.estimator.head.regression_head failed: Traceback (most recent call last):
  File "/home/amanda/.local/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/amanda/.local/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 450, in superreload
    update_generic(old_obj, new_obj)
  File "/home/amanda/.local/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 387, in update_generic
    update(a, b)
  File "/home/amanda/.local/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 357, in update_class
    update_instances(old, new)
  File "/home/amanda/.local/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 317, in update_instances
    update_instances(old, new, obj, visited)
  File "/home/amanda/.local/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 317, in update_instances
    

[autoreload of tensorflow_estimator._api.v1.estimator.tpu failed: Traceback (most recent call last):
  File "/home/amanda/.local/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/amanda/.local/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 434, in superreload
    module = reload(module)
  File "/usr/lib/python3.6/imp.py", line 315, in reload
    return importlib.reload(module)
  File "/usr/lib/python3.6/importlib/__init__.py", line 141, in reload
    name = module.__spec__.name
  File "/home/amanda/.local/lib/python3.6/site-packages/tensorflow_core/python/util/module_wrapper.py", line 192, in __getattr__
    attr = getattr(self._tfmw_wrapped_module, name)
  File "/home/amanda/.local/lib/python3.6/site-packages/tensorflow_core/python/util/module_wrapper.py", line 192, in __getattr__
    attr = getattr(self._tfmw_wrapped_module, name)
  File "/home/amanda/.local/lib/python3.6/s

[autoreload of tensorflow_estimator.python.estimator.canned.dnn_linear_combined failed: Traceback (most recent call last):
  File "/home/amanda/.local/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/amanda/.local/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 450, in superreload
    update_generic(old_obj, new_obj)
  File "/home/amanda/.local/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 387, in update_generic
    update(a, b)
  File "/home/amanda/.local/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 357, in update_class
    update_instances(old, new)
  File "/home/amanda/.local/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 317, in update_instances
    update_instances(old, new, obj, visited)
  File "/home/amanda/.local/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 317, in update_instance

[autoreload of tensorflow_estimator.python.estimator.api failed: Traceback (most recent call last):
  File "/home/amanda/.local/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/amanda/.local/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 434, in superreload
    module = reload(module)
  File "/usr/lib/python3.6/imp.py", line 315, in reload
    return importlib.reload(module)
  File "/usr/lib/python3.6/importlib/__init__.py", line 141, in reload
    name = module.__spec__.name
  File "/home/amanda/.local/lib/python3.6/site-packages/tensorflow_core/python/util/module_wrapper.py", line 192, in __getattr__
    attr = getattr(self._tfmw_wrapped_module, name)
  File "/home/amanda/.local/lib/python3.6/site-packages/tensorflow_core/python/util/module_wrapper.py", line 192, in __getattr__
    attr = getattr(self._tfmw_wrapped_module, name)
  File "/home/amanda/.local/lib/python3.6/si

In [32]:
from keras.utils.vis_utils import plot_model
model = Sequential()
model.add(Dense(2, input_dim=1, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

[autoreload of tensorflow.summary failed: Traceback (most recent call last):
  File "/home/amanda/.local/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/amanda/.local/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 434, in superreload
    module = reload(module)
  File "/usr/lib/python3.6/imp.py", line 315, in reload
    return importlib.reload(module)
  File "/usr/lib/python3.6/importlib/__init__.py", line 141, in reload
    name = module.__spec__.name
  File "/home/amanda/.local/lib/python3.6/site-packages/tensorflow/__init__.py", line 50, in __getattr__
    module = self._load()
  File "/home/amanda/.local/lib/python3.6/site-packages/tensorflow/__init__.py", line 45, in _load
    self._parent_module_globals[self._local_name] = module
  File "/home/amanda/.local/lib/python3.6/site-packages/tensorflow/__init__.py", line 50, in __getattr__
    module = self._load()
  File "/

[autoreload of tensorflow_estimator.python.estimator.model_fn failed: Traceback (most recent call last):
  File "/home/amanda/.local/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/amanda/.local/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 450, in superreload
    update_generic(old_obj, new_obj)
  File "/home/amanda/.local/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 387, in update_generic
    update(a, b)
  File "/home/amanda/.local/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 357, in update_class
    update_instances(old, new)
  File "/home/amanda/.local/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 317, in update_instances
    update_instances(old, new, obj, visited)
  File "/home/amanda/.local/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 317, in update_instances
    update_insta

KeyboardInterrupt: 

In [ ]:
# https://machinelearningmastery.com/visualize-deep-learning-neural-network-model-keras/#:~:text=Summarize%20Model,output%20shape%20of%20each%20layer.

In [ ]:
model.evaluate(X_test, y_test, verbose=False)

In [ ]:
best, best_doce, best_agua, [agua_certo, doce_certo] = get_best(X_test, y_test, model)
print(best, best_doce, best_agua)
plt.plot(agua_certo)
plt.plot(doce_certo)
plt.plot(agua_certo + doce_certo)

In [ ]:
#model.save('04_06_acucar_sucralose.h5')

In [ ]:
X_test.shape

In [ ]:
model.evaluate(X_test, y_test, verbose=False)

In [ ]:
from kerastuner import HyperModel
from kerastuner.tuners import RandomSearch

def create_model_base(hp):
    model = tf.keras.Sequential()
    model.add(Dense(hp.Int('Dense_01', min_value=0, max_value=128, step=16,default=64), activation='relu'))
    model.add(Dropout(rate=hp.Float('dropout_1', min_value=0.0, max_value=0.4, default=0.25, step=0.05)))
    model.add(Dense(hp.Int('Dense_02', min_value=0, max_value=128, step=16,default=64), activation='relu'))
    model.add(Dropout(rate=hp.Float('dropout_2', min_value=0.0, max_value=0.4, default=0.25, step=0.05)))
    model.add(Dense(hp.Int('Dense_03', min_value=0, max_value=128, step=16,default=64), activation='relu'))
    model.add(Dropout(rate=hp.Float('dropout_3', min_value=0.0, max_value=0.4, default=0.25, step=0.05)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def create_tunned_model(hp):
    model = tf.keras.Sequential()  
    model.add(Dense(hp.Int('Dense_01', min_value=16, max_value=128, step=16,default=64), activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(hp.Int('Dense_02', min_value=16, max_value=128, step=16,default=64), activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
tuner = RandomSearch(create_tunned_model, objective='val_accuracy', max_trials=10, executions_per_trial=5, directory='log_dir_agua_aspartame3')

In [ ]:
tuner.search(x=X_train, y=y_train, epochs=200, batch_size=64, validation_data=(X_test, y_test))

In [ ]:
tuner.results_summary()

In [ ]:
best_model = tuner.get_best_models(num_models=3)[0]

In [ ]:
best_model.summary()

In [ ]:
loss, accuracy = best_model.evaluate(X_test, y_test)

In [ ]:
best, best_doce, best_agua, [agua_certo, doce_certo] = get_best()
print(best, best_doce, best_agua)
plt.plot(agua_certo)
plt.plot(doce_certo)
plt.plot(agua_certo + doce_certo)

In [ ]:
tf.keras.models.save_model(best_model, 'tunner01_agua_acucar_V1.h5')

In [ ]:
best_model.summary()

In [ ]:
model = return_model()

In [ ]:
model.fit(X_train, y_train, epochs=150, verbose=True, shuffle=False)  # validation_data=(X_test, y_test)

In [ ]:
#model = tf.keras.models.load_model('15_05_V3agua_acucar.h5')

In [ ]:
model.evaluate(X_test, y_test, verbose=False)

In [ ]:
print(calc_preds(0.95, X_test, y_test, model))

In [ ]:
def get_best():
    doce_certo, agua_certo = np.linspace(0, 1, num=101), np.linspace(0, 1, num=101)
    for i, j in enumerate(doce_certo):
        agua_certo[i], doce_certo[i] = calc_preds(j, X_test, y_test, model)
    b = agua_certo + doce_certo
    return b.argmax(), doce_certo[b.argmax()], agua_certo[b.argmax()], [agua_certo, doce_certo]

In [ ]:
best, best_doce, best_agua, [agua_certo, doce_certo] = get_best()
print(best, best_doce, best_agua)
plt.plot(agua_certo)
plt.plot(doce_certo)
plt.plot(agua_certo + doce_certo)

In [ ]:
#model.save('15_05_V3acucar_acucar.h5')